In [1]:
import sys
import gym
import numpy as np
from collections import defaultdict

Create an instance of OpenAI Gym Blackjack environment

In [2]:
env = gym.make('Blackjack-v0')

Each state is a 3-tuple of:
- the player's current sum $\in \{0, 1, \ldots, 31\}$,
- the dealer's face up card $\in \{1, \ldots, 10\}$, and
- whether or not the player has a usable ace (`no` $=0$, `yes` $=1$).

The agent has two potential actions:

```
    STICK = 0
    HIT = 1
```

In [3]:
def generate_episode_from_Q(env, Q, epsilon, nA):
    
    # 'episode' is a list that holds the [[state], [action], reward]
    episode = []
    state = env.reset()
    
    while True:
        
        # 'action' is obtained by implementing a stoichastic policy based on the greedily obtained Q-Table.
        # get_probs() will return a list with as many entries as possible actions in this environment.
        # in this case, nA = 2.
        # associated with that list are probabilities of choosing that particular index. 
        # for example, p=[0.33, 0.66], which indicates that there is a 0.33 probability to choose index 0
        # and a 0.66 probability of choosing index 1.
        # numpy will then take this information and return either 0 or 1, which corresponds to a stick(0) or 
        # a hit(1). 
        # Note: this only occurs if the Q-Table has an entry for the given state, denoted by
        # 'if state in Q'
        # else, an action is chosen at random.
        action = np.random.choice(np.arange(nA), p=get_probs(Q[state], epsilon, nA)) \
                                    if state in Q else env.action_space.sample()

        # play out the hand given the action. 
        # note that 'done' can be True due to the agent choosing to stay and winning/losing, from the agent 
        # choosing to hit and going over 21, or from the agent choosing to hit/stay and drawing. 
        next_state, reward, done, info = env.step(action)
        
        # append the state, action, and reward to the episode list.
        # note that the state is the prior state, not the next_state after having performed the action.
        # this piece of code wraps up the state, the action, and the result of that action.
        episode.append((state, action, reward))
        
        # advance the state.
        state = next_state
        
        # if the hand is over, exit the loop.
        if done:
            break
            
    # return the results of playing several hands of blackjack
    return episode

def get_probs(Q_s, epsilon, nA):
    
    # initialize a list with as many indices as there are actions in this environment.
    # for the case of blackjack, nA = 2
    # every entry is initialized with the same probability, epsilon / nA
    policy_s = np.ones(nA) * epsilon / nA
    
    # Q_s is a list of size 2 that contains expected rewards for a particular action.
    # e.g., Q_s = [0.87, -0.44]
    # This indicates that sticking(index 0) has an expected reward of 0.87 for this particular state.
    # on the other hand, hitting(index 1) has an expected reward of -0.44.
    # best_a will contain the INDEX of the largest value in the list.
    best_a = np.argmax(Q_s)
    
    # This line will increase the probability of choosing the action that corresponds to the greater
    # expected reward.
    # Note that we add to the equiprobably value of (epsilon/nA) an amount of (1-epsilon).
    # Early in training, epsilon is closer to 1, which results in a smaller increment to
    # the currently highest rewarded action.
    # This results in other actions have a greater chance of being executed, and thus the agent
    # will 'explore' rather than exploit.
    # On the other hand, as training increases, epsilon becomes smaller and the value (1-epsilon) grows
    # which increases the probability of choosing the greater expected reward -- i.e., the agent favors the
    # exploitation of its knowledge of the environment. 
    policy_s[best_a] = 1 - epsilon + (epsilon / nA)
    
    # summary: in the beginning of training, the policy may look like
    # [0.48, 0.44]
    # for a particular state.
    # toward the end of training, the policy may look like
    # [0.56, -0.81]
    return policy_s

def update_Q(env, episode, Q, alpha, gamma):
    
    # place the states, actions, and rewards into separate lists
    states, actions, rewards = zip(*episode)
    
    # initialize a list that contains discounted expected rewards.
    # discounting is not implemented here, but will be discussed/explained in more detail 
    # in later projects in this repo.
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    
    # this loop will update the Q-Table. 
    for i, state in enumerate(states):
        
        # get the current value in the for a given State/Action pair
        old_Q = Q[state][actions[i]] 
        
        # update the Q-Table using a variable alpha. 
        # alpha corresponds to the amount of influence the new reward has in updating the current Q-Table entry.
        # if alpha is large, then the new expected reward has a greater effect on the value in the Q-Table.
        # conversely, if alpha is small, then the new reward has a smaller impact on the value in the Q-Table.
        Q[state][actions[i]] = old_Q + alpha*(sum(rewards[i:]*discounts[:-(1+i)]) - old_Q)
        
    return Q

In [4]:
def mc_control(env, num_episodes, alpha, gamma=1.0, eps_start=1.0, eps_decay=.99999, eps_min=0.05):

    # number of actions in the action space = 2, for 'stick' and 'hit'
    nA = env.action_space.n
    
    # Q-Table will hold the expected reward for a given State/Action pair. It is a nexted dictionary.
    # Q[state][action] = reward
    Q = defaultdict(lambda: np.zeros(nA))
    
    # The value of epsilon determines the degree to which the Agent trusts its knowledge of the environment
    # when determining its next action. Early on, it is favorable for the Agent to explore various 
    # State/Action spaces to gain a more well-rounded understanding of its environment.
    epsilon = eps_start

    # num_episodes is equivalent to the number of hands of Blackjack that will be played.
    for i_episode in range(1, num_episodes+1):
        
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        # set the value of epsilon. Note the decay rate and that there is a limit to the minimum of 
        # epsilon. If epsilon becomes too low too fast, the agent will strictly favor exploitation.
        epsilon = max(epsilon*eps_decay, eps_min)
        
        # generate a list of episodes by following epsilon-greedy policy
        episode = generate_episode_from_Q(env, Q, epsilon, nA)
        
        # update the action-value function estimate using the episode
        Q = update_Q(env, episode, Q, alpha, gamma)
        
    # determine the policy corresponding to the final action-value function estimate.
    # 'policy' will contain a dictionary of States and Actions, 
    # e.g., [[13, 5, False], 1] --> given the agent has a sum of 13, the dealer is showing a card of value 5,
    # and the agent does not have a usable Ace, the best action is to hit.
    policy = dict((k,np.argmax(v)) for k, v in Q.items())
    
    return policy, Q

Generate a policy and Q-Table. 500,000 = number of episodes to play.

In [5]:
policy, Q = mc_control(env, 500000, 0.02)

Episode 500000/500000.